In [50]:
# Install dependencies
!pip install -U funasr flask-ngrok flask

# Import libraries
import os
import json
import numpy as np
from flask import Flask, request, jsonify
from funasr import AutoModel


In [51]:
# Load model globally (only once)
model = AutoModel(model="emotion2vec_plus_large")
print("✅ Model loaded successfully!")


funasr version: 1.2.7.
Check update of funasr, and it would cost few times. You may disable it by set `disable_update=True` in AutoModel
You are using the latest version of funasr-1.2.7


Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.0.0.weight, /root/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.0.0.bias, /root/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.1.0.weight, /root/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.1.0.bias, /root/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.2.0.weight, /root/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.2.0.bias, /root/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.3.0.weight, /root/.cache/modelscope/hub/models

In [52]:
app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    try:
        file = request.files["audio"]
        file_path = "temp.wav"
        file.save(file_path)

        # Run model
        result = model.generate(file_path, granularity="utterance")
        data = result[0]

        if 'predictions' in data and 'scores' in data:
            emotions = data['predictions']
            scores = data['scores']
        elif 'labels' in data and 'scores' in data:
            emotions = [e.split('/')[-1] for e in data['labels']]
            scores = data['scores']
        else:
            return jsonify({"error": "No 'labels' or 'predictions' found"}), 500

        best_idx = int(np.argmax(scores))
        best_emotion = emotions[best_idx]
        confidence = float(scores[best_idx])
        top_emotions = sorted(zip(emotions, scores), key=lambda x: x[1], reverse=True)
        top_emotions_json = [{"emotion": e, "score": float(s)} for e, s in top_emotions]

        return jsonify({
            "emotion": best_emotion,
            "confidence": confidence,
            "topEmotions": top_emotions_json
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [56]:
# Install pyngrok
!pip install pyngrok
from pyngrok import ngrok

# Set ngrok authtoken
# Replace "YOUR_AUTH_TOKEN" with your actual ngrok authentication token
ngrok.set_auth_token("31y3EqKzzZqEmFcksJJiO0jFShJ_76ywiqbQqegsSHLULmtL")

# Start tunnel
public_url = ngrok.connect(5001)
print("🔗 Public URL:", public_url)

# Run Flask app
app.run(port=5001)

🔗 Public URL: NgrokTunnel: "https://aecd474dfb30.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
rtf_avg: 5.669: 100%|██████████| 1/1 [00:11<00:00, 11.35s/it]
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 16:32:18] "POST /predict HTTP/1.1" 200 -
rtf_avg: 3.587: 100%|██████████| 1/1 [00:07<00:00,  7.18s/it]
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 16:36:44] "POST /predict HTTP/1.1" 200 -
rtf_avg: 3.151: 100%|██████████| 1/1 [00:06<00:00,  6.31s/it]
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 16:39:00] "POST /predict HTTP/1.1" 200 -
rtf_avg: 0.621: 100%|██████████| 1/1 [00:04<00:00,  4.11s/it]
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 16:55:58] "POST /predict HTTP/1.1" 200 -


In [36]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mv ngrok /usr/local/bin/ngrok


--2025-08-29 16:08:39--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 75.2.60.68, 99.83.220.108, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.28M  18.6MB/s    in 0.7s    

2025-08-29 16:08:40 (18.6 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [37]:
!ngrok authtoken 31y3EqKzzZqEmFcksJJiO0jFShJ_76ywiqbQqegsSHLULmtL


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [38]:
# Step 5: Create Flask app
app = Flask(__name__)
from flask_cors import CORS
CORS(app)  # optional, allows requests from any origin

@app.route("/predict", methods=["POST"])
def predict():
    try:
        file = request.files["audio"]
        file_path = "temp.wav"
        file.save(file_path)

        # Run inference
        result = model.generate(file_path, granularity="utterance")

        # Flexible extraction of labels
        data = result[0]
        if 'predictions' in data and 'scores' in data:
            emotions = data['predictions']
            scores = data['scores']
        elif 'labels' in data and 'scores' in data:
            emotions = [e.split('/')[-1] for e in data['labels']]  # remove Chinese
            scores = data['scores']
        else:
            return jsonify({"error": "No 'labels' or 'predictions' found"}), 500

        best_idx = int(np.argmax(scores))
        best_emotion = emotions[best_idx]
        confidence = float(scores[best_idx])
        top_emotions = sorted(zip(emotions, scores), key=lambda x: x[1], reverse=True)
        top_emotions_json = [{"emotion": e, "score": float(s)} for e, s in top_emotions]

        return jsonify({
            "emotion": best_emotion,
            "confidence": confidence,
            "topEmotions": top_emotions_json
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [39]:
def run_app():
    app.run(port=5000)

threading.Thread(target=run_app).start()


 * Serving Flask app '__main__'


In [48]:
!which ngrok


/usr/local/bin/ngrok


In [49]:
import subprocess
import time
import requests

# Start ngrok
ngrok_path = "/usr/local/bin/ngrok"
port = "5000"

# Kill any previous ngrok sessions
subprocess.run(["pkill", "-f", "ngrok"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Start ngrok
ngrok = subprocess.Popen([ngrok_path, "http", port])
print(f"Started ngrok with PID {ngrok.pid}")

# Wait for ngrok to initialize
time.sleep(5)

# Fetch the public URL
try:
    response = requests.get("http://127.0.0.1:4040/api/tunnels")
    tunnels = response.json()["tunnels"]
    public_url = tunnels[0]["public_url"]
    print("Your public URL:", public_url)
except Exception as e:
    print("Error getting public URL:", e)


Started ngrok with PID 38797
Error getting public URL: HTTPConnectionPool(host='127.0.0.1', port=4040): Max retries exceeded with url: /api/tunnels (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d0ac503b230>: Failed to establish a new connection: [Errno 111] Connection refused'))
